In [ ]:
import pandas as _hex_pandas
import datetime as _hex_datetime
import json as _hex_json

In [ ]:
hex_scheduled = _hex_json.loads("false")

In [ ]:
hex_user_email = _hex_json.loads("\"example-user@example.com\"")

In [ ]:
hex_run_context = _hex_json.loads("\"logic\"")

In [ ]:
hex_timezone = _hex_json.loads("\"UTC\"")

In [ ]:
hex_project_id = _hex_json.loads("\"c924e572-5743-45ad-81e2-c6e5bc30e579\"")

In [ ]:
hex_project_name = _hex_json.loads("\"Salon/Barbershop Density by City/Neighborhood/Zipcode\"")

In [ ]:
hex_status = _hex_json.loads("\"\"")

In [ ]:
hex_categories = _hex_json.loads("[]")

In [ ]:
hex_color_palette = _hex_json.loads("[\"#4C78A8\",\"#F58518\",\"#E45756\",\"#72B7B2\",\"#54A24B\",\"#EECA3B\",\"#B279A2\",\"#FF9DA6\",\"#9D755D\",\"#BAB0AC\"]")

#### This SQL script processes and analyzes data for voter engagement efforts in major cities within swing states in the USA, with a focus on barbershops and salons. The primary goal is to identify high-priority areas within these cities by examining postal codes, shop locations, and demographic information, and then assessing the adequacy of scraping coverage in each locality. 

# Breakdown of Salons/Barbershops by State

Arizona (AZ): 8,875\
Georgia (GA): 13,656\
Nevada (NV): 3,422\
Michigan (MI): 11,492\
North Carolina (NC): 16,200\
Pennsylvania (PA): 20,322\
Wisconsin (WI): 8,637\
Ohio (OH): 15,647\
Colorado (CO): 6,661\
Florida (FL): 40,060\
Texas (TX): 32,578\
Virginia (VA): 7,630\
New Hampshire (NH): 1,625 (estimated)

**Est. Number of Salons/Barbershops in Swing States = 186, 805**

[Reference: SalonSpaConnection.com Hair Salon Industry Statistics](https://salonspaconnection.com/beauty-hair-salon-industry-statistics-in-2023/)

\
**Source Note**: *The hair salon industry in the US includes approximately 444,102 licensed salons, spas and barber shops in 2023. Our reports reflect approximate numbers for states that were unreachable for this information. We gathered data from each state for this report, the census and the Professional Beauty Association, (PBA), had incorrect reporting per state.* 

**Researcher Note (Stann)**: *Some states were uncooperative or had inaccessible information, which for our project, specifically affected the estimates for New Hampshire. Additionally, our analysis primarily focuses on registered salons and barbershops. It’s important to note that in many communities, particularly communities of color and working-class areas, there is a significant likelihood that hair stylists and other beauty professionals may not have a license or operate from an official storefront.*

In [ ]:
# import jinja2
# raw_query = """
#     
#     with raw_bing as (
#     select distinct 
#       regexp_replace(PhoneNumber, r'\D', '') as id, 
#       left(Address_postalCode, 5) as Address_postalCode,
#       * except (Address_postalCode)
#     from sutv.raw_bing_api 
#     where PhoneNumber is not null 
#     and Address_postalCode is not null
#     and Address_postalCode != 'Null'
#     ),
#     
#     sutv_leads as (
#       select distinct left(zip_code, 5) as zip_code, * except (zip_code) 
#       from sutv.sutv_leads
#       where zip_code != 'nan'
#     ),
#     
#     raw_zips as (
#       select lpad(zipcode, 5, '0') as zipcode, * except (zipcode)
#       from sutv.raw_zips
#     ),
#     
#     mapped as (
#       select
#         coalesce(b.id, s.id) as id,
#         upper(coalesce(s.state_abbreviation, b.Address_adminDistrict))  as state,
#         coalesce(s.osm_id, b.id) as osm_id,
#         s.housenumber, 
#         s.street,
#         s.shop_type,
#         b.id as bing_id, 
#         s.id as mailable_id, 
#         z.zipcode as target_zipcode, 
#         s.zip_code as mailable_zipcode,
#         b.Address_postalCode as bing_zipcode,
#         case when s.id is not null then 'Mailable Address' else 'Non-Mailable Address' end as is_mailable, 
#         case when z.high_priority_zip = 'True' then 'Target Zip' else 'Non-Target Zip' end as is_target
#       from raw_bing b 
#       full outer join sutv_leads s 
#         on s.id = b.id
#       left join raw_zips z
#         on trim(cast(z.zipcode as string)) = coalesce(trim(cast(s.zip_code as string)), trim(cast(b.Address_postalCode as string)))
#       where 1=1 
#         and (z.zipcode is not null or s.zip_code is not null or b.Address_postalCode is not null)
#     ),
#     
#     target_states as (
#       select *
#       from mapped
#       where state in ('TX','FL','NC','AZ','GA','OH','PA','MI','WI','NV', 'NH')
#     ),
#     
#     deduped as (
#       select * 
#       from target_states
#       qualify row_number() over (partition by id order by mailable_id is not null desc, shop_type = 'barber' desc) = 1
#       -- qualify row_number() over (partition by osm_id, housenumber, street,shop_type order by mailable_id is not null desc, shop_type = 'barber' desc) = 1
#     ), 
#     
#     by_state  as (
#       select 
#         state,
#         count(1),
#         count(distinct id)
#       from deduped
#       group by 1
#       order by 2 desc
#     ) 
#     
#     select * from (
#       select 
#         is_mailable,
#         is_target,
#         count(1) as num_shops
#       from deduped 
#       group by 1,2
#     ) sub
#     pivot (
#       sum(num_shops) for is_mailable in ('Mailable Address', 'Non-Mailable Address')
#     )
#     
# """
# sql_query = jinja2.Template(raw_query).render(vars())

# CITY ANALYSIS

Researcher's Note (Stann): This analysis focuses on urban areas within the most populous cities in each state, specifically targeting the main city and its surrounding suburbs. It is important to distinguish these urban areas from larger regions such as the Greater Area, Metropolitan Statistical Areas (MSA), or Combined Statistical Areas (CSA), as these broader classifications can include suburbs from neighboring states. Including such regions could skew our state-specific benchmarks. For example, the Greater Philadelphia area might encompass parts of New Jersey, Maryland, and Delaware, none of which are swing states.

# US Industry Benchmark

A common industry benchmark in the US is approximately 1 salon/barbershop per 1,500 to 2,000 people. I consider this benchmark too broad based on the granularity of the data (city and district-level) we're looking at. This is because it might undercount the number of barbershops per capita in urban areas due to the weighting of rural/suburban localities.

## Case Study of Philadelphia

Philadelphia has a population of about 1.6 million people. The city's diverse population might lead to a higher demand for specialized hair services, potentially requiring more salons and barbershops. The local economy, average income levels, and urban density can influence the number of viable salons and barbershops.

Based on the industry benchmark, I estimate the following thresholds if we assume that for every 1,500 or 2,000 people, there is 1 salon/barbershop:

**Low Estimate**: 1 salon/barbershop per 2,000 people → 1,600,000 / 2,000 = 800 salons/barbershops.\
**High Estimate**: 1 salon/barbershop per 1,500 people → 1,600,000 / 1,500 = 1,067 salons/barbershops.

As noted, the density of barbershops may be affected by geographic location: 

**Urban Areas**: Higher density areas may support more salons per capita due to ease of access and higher foot traffic.\
**Affluent Areas**: Higher income neighborhoods might have more demand for premium services, increasing the number of salons and barbershops.

As a result, based on the conservative industry benchmark, there should be approximately 800 to 1,100 salons/barbershops in Metro Philadelphia.

On the other hand, the Philadelphia Urban Area (PUA) with 5.7M residents, will have between 2850 to 3800 salons/barbershops.

Note: *The Philadelphia Urban Area is NOT the same as the Greater Philadelphia area, which has 6M+ residents and includes parts of New Jersey and Delaware.*

**Takeaway**: Our dataset has around 2250 barbershops across the PUA. As a result, we don't have a representative sample of salons on a city or district level. 

# National Average Benchmark

According to SalonSpaConnection.com, in the USA, there are 444,102 salons (ca. 2023) in a population of 343,477,335. Because of the previously mentioned limitations of the industry benchmark, I will use the national average benchmark (1:773) for understanding the overall market saturation. Since Philadephia is probably a very saturated (urban) city, it will probably align closer with the national average. Due to its large Black population, I would assume the ratio is lower than the national average benchmark. A lower ratio (i.e., more salons per person) could indicate market saturation, especially in urban areas where competition is higher. 

Conversely, in less densely populated areas, the ratio might be higher (fewer salons per person). The large gap between these numbers could indicate areas where the market is very competitive (closer to the national average) versus those that are still developing (closer to the industry benchmark). On a district level, I will roughly expect ratios to be lower in more urban neighbourhoods and higher in less urban neighborhoods.

Therefore, based on the national average benchmark, with a population of 1.6M, Metro Philadelphia should have around 2070 salons, while, the Philadelphia urban area with a population of 5.7M should have around 7369 salons. 

# Creating a State-specific Benchmark after Double Checking Assumptions based on Total Population of Pennsylvania

According to Salon Spa Connection, there are 20,322 salons (ca. 2023) in the entire state of Pennsylvania. 

Step 1: Calculate the Total Number of Salons/Barbershops in Pennsylvania:

- With a population of 12,961,683, this is a ratio of 1:638. This is lower than the national average density, 1:773.

Step 2: Determine Population Share of PUA:

- 5,700,000/12,800,000 = 0.445 (or 44.5%)

Step 3: Estimate the Number of Salons/Barbershops in PUA Based on its Share of State Population

- Number of Salons/Barbershops in Philly = 0.445*20,322 = 9043

After double-checking my assumptions, we can see that based on the state average of 1:638, **Philadelphia has even more salon/barbershops than expected**. 

Now, we can be even more precise. **Thus, the state average benchmark for salons per person in Pennsylvania is 1:633**. This benchmark takes into account state-wide density trends that are obscured by the national average. 

**Takeaway**: Instead of using the static national benchmark of 1:773, we can use more accurate density benchmarks for each US state.

[Reference: SalonSpaConnection.com Hair Salon Industry Statistics](https://salonspaconnection.com/beauty-hair-salon-industry-statistics-in-2023/)

## Creating Service Density KPI to Measure Salon/Barbershop Coverage in Each Locality

To understand the district-level trends in salon density in Philadelphia, I am going to create a ratio KPI between 0 to 1 that measures the density of salons/barbershops in a given locality. 

**Create Service Density KPI: ratio_people_per_salon (normalized to a scale between 0 and 1)**

Assumptions (these numbers are just illustrative and do not correspond with the actual data): 

- Population: 1,600,000 people
- Number of Salons/Barbershops: 1,200

Step 1: Calculate the Ratio of People per Salon/Barbershop
- Calculate the people per salon/barbershop: 1,600,000 ÷ 1,200 = 1,333.33. This means there is 1 barbershop for every 1,333 people.

Step 2: Compare the Actual Ratio to the Benchmark
- Divide the consequent (second term in the ratio) in the national benchmark by the consequent in the actual ratio: 773 ÷ 1,333.33 = 0.58.

Step 3: Calculate the Normalized KPI
- Determine the final KPI: Since 0.58 is less than 1, the KPI is 0.58. The KPI of 0.58 indicates that the area has fewer salons/barbershops per capita than the national average benchmark ratio of 1:773. This suggests that we need to scrape for more salons/barbershops to meet the ideal ratio.
- If the result is greater than 1, this means that we have met or exceeded the benchmark ratio of 1 salon/barbershop per 773 people. This means coverage is adequate and we have scraped enough salons/barbershops in this district. In other words, we have an optimal number of salons/barbershops relative to the area's population.

In [ ]:
# import jinja2
# raw_query = """
#     with deduped_leads as (
#         select *
#         from sutv.sutv_leads 
#         qualify row_number() over (partition by osm_id order by zip_code desc) = 1
#     ),
#     shops_with_zip_priority as (
#         select 
#             d.*, z.*  from deduped_leads  d
#         left join sutv.raw_zips z
#             on cast(z.zipcode as string) = cast(d.zip_code as string)
#         where lower(city) = 'philadelphia'
#     ),
#     
#     -- select * 
#     -- from shops_with_zip_priority
#     -- where zipcode is null 
#     
#     top_level as (
#     select 
#         locality,
#         high_priority_zip,
#         population_by_year_2018,
#         avg(cast(replace(population_by_race_nonwhite_percentage, 'nan', '') as decimal)*100) as avg_nonwhite_pop,
#         count(distinct zip_code) as num_zips,
#         count(1) as num_shops,
#         -- Calculate the ratio of people per salon
#         (638 / (cast(population_by_year_2018 as decimal) / cast(count(1) as decimal))) as norm_people_per_salon
#     from shops_with_zip_priority
#     -- where high_priority_zip = 'False'
#     -- where 4 is not null
#     group by 1,2,3
#     order by 7 desc,3 desc
#     )
#     
#     select * 
#     from top_level;
#     
#     -- -- select sum(num_shops)
#     -- -- from top_level_philly
#     
#     -- -- select 
#     -- --     high_priority_zip,
#     -- --     count(1)
#     -- -- from shops_with_zip_priority 
#     -- -- group by 1 
# """
# sql_query = jinja2.Template(raw_query).render(vars())

In [ ]:
import altair
chart_dataframe_2 = altair.Chart.from_json("""
{
    "width": "container",
    "height": "container",
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "layer": [
        {
            "description": "outer data layer",
            "resolve": {
                "scale": {
                    "color": "independent",
                    "y": "shared"
                }
            },
            "layer": [
                {
                    "description": "bar series layer",
                    "transform": [],
                    "layer": [
                        {
                            "description": "bar mark layer",
                            "mark": {
                                "type": "bar",
                                "clip": true,
                                "filled": true,
                                "cursor": "pointer",
                                "orient": "vertical"
                            },
                            "encoding": {
                                "opacity": {
                                    "value": 1
                                },
                                "tooltip": [
                                    {
                                        "field": "locality",
                                        "type": "ordinal",
                                        "title": "locality"
                                    },
                                    {
                                        "field": "avg_nonwhite_pop",
                                        "type": "quantitative",
                                        "aggregate": "sum",
                                        "formatType": "NUMBER_FORMATTER",
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "$",
                                            "nanFormat": ""
                                        },
                                        "title": "Sum of avg_nonwhite_pop"
                                    }
                                ],
                                "color": {
                                    "value": "#4C78A8"
                                },
                                "x": {
                                    "field": "locality",
                                    "type": "ordinal",
                                    "title": "locality",
                                    "scale": {},
                                    "axis": {
                                        "grid": true,
                                        "ticks": true,
                                        "labels": true,
                                        "labelFlush": false,
                                        "labelOverlap": "greedy"
                                    }
                                },
                                "y": {
                                    "field": "avg_nonwhite_pop",
                                    "type": "quantitative",
                                    "aggregate": "sum",
                                    "title": "Sum of avg_nonwhite_pop",
                                    "scale": {},
                                    "axis": {
                                        "grid": true,
                                        "ticks": true,
                                        "labels": true,
                                        "labelFlush": false,
                                        "format": {
                                            "format": "NUMBER",
                                            "columnType": "NUMBER",
                                            "numDecimalDigits": -1,
                                            "currency": "$",
                                            "nanFormat": ""
                                        },
                                        "formatType": "NUMBER_FORMATTER"
                                    }
                                }
                            }
                        }
                    ],
                    "encoding": {
                        "x": {
                            "field": "locality",
                            "type": "ordinal",
                            "title": "locality",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "labelOverlap": "greedy"
                            }
                        },
                        "y": {
                            "field": "avg_nonwhite_pop",
                            "type": "quantitative",
                            "aggregate": "sum",
                            "title": "Sum of avg_nonwhite_pop",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "format": {
                                    "format": "NUMBER",
                                    "columnType": "NUMBER",
                                    "numDecimalDigits": -1,
                                    "currency": "$",
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            }
                        }
                    }
                }
            ],
            "transform": []
        }
    ],
    "config": {
        "legend": {
            "orient": "right"
        },
        "font": "\"IBM Plex Sans\", system-ui, -apple-system, BlinkMacSystemFont, sans-serif",
        "view": {}
    },
    "datasets": {
        "layer00": [
            {
                "name": "dummy",
                "value": 0
            }
        ]
    },
    "usermeta": {
        "selectionConfigs": {},
        "columnNameMappings": {}
    }
}
""")
chart_dataframe_2.datasets.layer00 = dataframe_2.to_json(orient='records')
chart_dataframe_2.display(actions=False)

# Creating a State Benchmark for Georgia

According to Salon Spa Connection, there are 13,656 salons (ca. 2023) in the entire state of Georgia. 

Step 1: Calculate the Total Number of Salons/Barbershops in Georgia:

- With a population of 11,029,227 (ca. 2023), this is a ratio of 1:807. This is slightly higher than the national average density, 1:773.

Step 2: Determine Population Share of Greater ATL:

- 6,307,261/11,029,227 = 0.572 (or 57.2%)

Step 3: Estimate the Number of Salons/Barbershops in PUA Based on its Share of State Population

- Est. Number of Salons/Barbershops in Greater ATL = 0.572*13,656 = 7,811

After double-checking my assumptions, we can see that based on the state average of 1:807, **Georgia has slightly less salon/barbershops than national average**. 

[Reference: SalonSpaConnection.com Hair Salon Industry Statistics](https://salonspaconnection.com/beauty-hair-salon-industry-statistics-in-2023/)

In [ ]:
# import jinja2
# raw_query = """
#     with deduped_leads as (
#         select distinct *
#         from sutv.sutv_leads 
#         qualify row_number() over (partition by osm_id order by zip_code desc) = 1
#     ),
#     shops_with_zip_priority as (
#         select  distinct 
#             d.*, z.*  from deduped_leads  d
#         left join sutv.raw_zips z
#             on cast(z.zipcode as string) = cast(d.zip_code as string)
#         where lower(city) = 'atlanta'
#     ),
#     -- select * 
#     -- from shops_with_zip_priority
#     -- where zipcode is null 
#     top_level as (
#     select 
#         locality,
#         high_priority_zip,
#         population_by_year_2018,
#         avg(safe_cast(replace(population_by_race_nonwhite_percentage, 'nan', '') as decimal)) as avg_nonwhite_pop,
#         count(distinct zip_code) as num_zips,
#         count(1) as num_shops,
#         -- Calculate the ratio of people per salon - 773 is the natl avg
#         (807 / (safe_cast(population_by_year_2018 as decimal) / safe_cast(count(1) as decimal))) as norm_people_per_salon
#     from shops_with_zip_priority
#     -- where high_priority_zip = 'False'
#     -- where 4 is not null
#     group by 1,2 ,3
#     order by 7 desc,3 desc
#     )
#     select * 
#     from top_level;
#     
#     -- -- select sum(num_shops)
#     -- -- from top_level_philly
#     
#     -- -- select 
#     -- --     high_priority_zip,
#     -- --     count(1)
#     -- -- from shops_with_zip_priority 
#     -- -- group by 1 
# """
# sql_query = jinja2.Template(raw_query).render(vars())

# Creating a State Benchmark for Michigan

According to Salon Spa Connection, there are 11,492 salons (ca. 2023) in the entire state of Michigan. 

Step 1: Calculate the Total Number of Salons/Barbershops in Michigan:

- With a population of 10,037,261 (ca. 2023), this is a ratio of 1:873. This is higher than the national average density, 1:773.

Step 2: Determine Population Share of Detroit Urban Area (DUA):

- 3,521,000/10,037,261 = 0.351 (or 35.1%)

Step 3: Estimate the Number of Salons/Barbershops in DUA Based on its Share of State Population

- Est. Number of Salons/Barbershops in Detroit = 0.351*11,492 = 4034

After double-checking my assumptions, we can see that based on the state average of 1:873, **Michigan has less salon/barbershops than national average**. 

[Reference: SalonSpaConnection.com Hair Salon Industry Statistics](https://salonspaconnection.com/beauty-hair-salon-industry-statistics-in-2023/)

In [ ]:
# import jinja2
# raw_query = """
#     with deduped_leads as (
#         select distinct *
#         from sutv.sutv_leads 
#         qualify row_number() over (partition by osm_id order by zip_code desc) = 1
#     ),
#     shops_with_zip_priority as (
#         select  distinct 
#             d.*, z.*  from deduped_leads  d
#         left join sutv.raw_zips z
#             on cast(z.zipcode as string) = cast(d.zip_code as string)
#         where lower(city) = 'detroit'
#     ),
#     -- select * 
#     -- from shops_with_zip_priority
#     -- where zipcode is null 
#     top_level as (
#     select 
#         locality,
#         high_priority_zip,
#         population_by_year_2018,
#         avg(safe_cast(replace(population_by_race_nonwhite_percentage, 'nan', '') as decimal)) as avg_nonwhite_pop,
#         count(distinct zip_code) as num_zips,
#         count(1) as num_shops,
#         -- Calculate the ratio of people per salon - 773 is the natl avg
#         (873 / (safe_cast(population_by_year_2018 as decimal) / safe_cast(count(1) as decimal))) as norm_people_per_salon
#     from shops_with_zip_priority
#     -- where high_priority_zip = 'False'
#     -- where 4 is not null
#     group by 1,2 ,3
#     order by 7 desc,3 desc
#     )
#     select * 
#     from top_level;
#     
#     -- -- select sum(num_shops)
#     -- -- from top_level_philly
#     
#     -- -- select 
#     -- --     high_priority_zip,
#     -- --     count(1)
#     -- -- from shops_with_zip_priority 
#     -- -- group by 1 
# """
# sql_query = jinja2.Template(raw_query).render(vars())

# Creating a State Benchmark for North Carolina

According to Salon Spa Connection, there are 16,200 salons (ca. 2023) in the entire state of North Carolina. 

Step 1: Calculate the Total Number of Salons/Barbershops in NC:

- With a population of 10,835,491 (ca. 2023), this is a ratio of 1:669. This is higher than the national average density, 1:773.

Step 2: Determine Population Share of Charlotte Urban Area (CUA):

- 1,379,873/10,835,491 = 0.127 (or 12.7%)

Step 3: Estimate the Number of Salons/Barbershops in DUA Based on its Share of State Population

- Est. Number of Salons/Barbershops in Charlotte Urban Area = 0.127*16,200 = 2057

After double-checking my assumptions, we can see that based on the state average of 1:669, **North Carolina has more salon/barbershops than national average**. 

[Reference: SalonSpaConnection.com Hair Salon Industry Statistics](https://salonspaconnection.com/beauty-hair-salon-industry-statistics-in-2023/)

In [ ]:
# import jinja2
# raw_query = """
#     with deduped_leads as (
#         select distinct *
#         from sutv.sutv_leads 
#         qualify row_number() over (partition by osm_id order by zip_code desc) = 1
#     ),
#     shops_with_zip_priority as (
#         select  distinct 
#             d.*, z.*  from deduped_leads  d
#         left join sutv.raw_zips z
#             on cast(z.zipcode as string) = cast(d.zip_code as string)
#         where lower(city) = 'charlotte'
#     ),
#     -- select * 
#     -- from shops_with_zip_priority
#     -- where zipcode is null 
#     top_level as (
#     select 
#         locality,
#         high_priority_zip,
#         population_by_year_2018,
#         avg(safe_cast(replace(population_by_race_nonwhite_percentage, 'nan', '') as decimal)) as avg_nonwhite_pop,
#         count(distinct zip_code) as num_zips,
#         count(1) as num_shops,
#         -- Calculate the ratio of people per salon - 773 is the natl avg
#         (873 / (safe_cast(population_by_year_2018 as decimal) / safe_cast(count(1) as decimal))) as norm_people_per_salon
#     from shops_with_zip_priority
#     -- where high_priority_zip = 'False'
#     -- where 4 is not null
#     group by 1,2 ,3
#     order by 7 desc,3 desc
#     )
#     select * 
#     from top_level;
#     
#     -- -- select sum(num_shops)
#     -- -- from top_level_philly
#     
#     -- -- select 
#     -- --     high_priority_zip,
#     -- --     count(1)
#     -- -- from shops_with_zip_priority 
#     -- -- group by 1 
# """
# sql_query = jinja2.Template(raw_query).render(vars())